In [1]:
pip install requests beautifulsoup4 pymongo pandas matplotlib 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt 

In [3]:
url = 'https://www.investing.com/indices/major-indices'
response = requests.get(url)
page_content = response.content 

In [4]:
soup = BeautifulSoup(page_content, 'html.parser')
table = soup.find('tbody', {'class': 'datatable-v2_body__8TXQk'})
rows = table.find_all('tr')

In [7]:
data = []
for row in rows[1:]: # Пропускаем заголовок
 cols = row.find_all('td')
 cols = [ele.text.strip() for ele in cols]  
 data.append(cols)
df = pd.DataFrame(data, columns=['#', 'Name', 'Last', 'High', 'Low', 'Change', 'Change%', 'Time'])
df['Last'] = df['Last'].str.replace(',', '').astype(float) 
df['High'] = df['High'].str.replace(',', '').astype(float) 
df['Low'] = df['Low'].str.replace(',', '').astype(float) 

In [8]:
df.head()

,#,Name,Last,High,Low,Change,Change%,Time
0,,S&P 500 derived,5485.31,5493.30,5441.81,+14.26,+0.26%,14:30:41
1,,Nasdaq,16996.41,17018.75,16801.13,+111.80,+0.66%,14:30:23
2,,Small Cap 2000 derived,2089.75,2099.15,2068.95,-8.03,-0.38%,14:30:25
3,,S&P 500 VIX derived,19.22,20.74,19.01,-0.23,-1.18%,14:30:31
4,,S&P/TSX,22879.28,23030.65,22791.55,-147.87,-0.64%,14:30:01


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   #        46 non-null     object 
 1   Name     46 non-null     object 
 2   Last     46 non-null     float64
 3   High     46 non-null     float64
 4   Low      46 non-null     float64
 5   Change   46 non-null     object 
 6   Change%  46 non-null     object 
 7   Time     46 non-null     object 
dtypes: float64(3), object(5)
memory usage: 3.0+ KB


In [10]:
conn = sqlite3.connect('financial_data.db')
df.to_sql('daily_ind', conn, if_exists='replace', index=False)

46

In [11]:
query = "SELECT * FROM daily_ind WHERE Name = 'Nasdaq'"
df_sqlite = pd.read_sql(query, conn)
print(df_sqlite) 

  #    Name      Last      High       Low   Change Change%      Time
0    Nasdaq  16996.41  17018.75  16801.13  +111.80  +0.66%  14:30:23


In [13]:
query = "SELECT * FROM daily_ind WHERE Last > 100000"
df_sqlite = pd.read_sql(query, conn)
print(df_sqlite) 

  #     Name      Last      High       Low Change Change%      Time
0    Bovespa  134135.0  134738.0  133754.0   -602  -0.45%  14:15:30


In [16]:
query = "SELECT * FROM daily_ind WHERE Last > 5000 AND High < 10000"
df_sqlite = pd.read_sql(query, conn)
print(df_sqlite) 

  #              Name     Last     High      Low  Change Change%      Time
0     S&P 500 derived  5485.31  5493.30  5441.81  +14.26  +0.26%  14:30:41
1    FTSE 100 derived  8205.98  8271.12  8195.13  -64.86  -0.78%  11:35:29
2              CAC 40  7407.55  7476.54  7386.53  -17.71  -0.24%  11:35:59
3                 PSI  6706.48  6809.30  6706.48  -68.41  -1.01%  11:35:59
4            BIST 100  9624.04  9788.36  9588.02  -75.52  -0.78%  11:09:59
5         S&P/ASX 200  8011.90  8057.90  7988.10  +23.80  +0.30%  02:04:59
6      SZSE Component  8073.83  8113.69  7973.79  +10.56  +0.13%  03:44:59
7       IDX Composite  7761.39  7764.78  7723.30  +58.65  +0.76%  03:59:59
8      PSEi Composite  6984.63  7109.75  6984.63   +0.38  +0.01%  02:58:59


In [20]:
query = "SELECT * FROM daily_ind WHERE High < 1000 ORDER BY High, Low"
df_sqlite = pd.read_sql(query, conn)
print(df_sqlite) 

  #                 Name    Last    High     Low  Change Change%      Time
0    S&P 500 VIX derived   19.22   20.74   19.01   -0.23  -1.18%  14:30:31
1         DJ New Zealand  351.82  353.72  351.04   +0.39  +0.11%  01:39:56
2            DJ Shanghai  371.96  372.91  368.19   +0.27  +0.07%  02:59:55
3                    AEX  881.84  890.91  880.22   -6.47  -0.73%  11:35:59
4                   RTSI  924.82  951.40  922.13  -17.32  -1.84%  11:40:00


In [22]:
query = "SELECT * FROM daily_ind WHERE Last = Low ORDER BY High, Low"
df_sqlite = pd.read_sql(query, conn)
print(df_sqlite) 

  #            Name     Last     High      Low  Change Change%      Time
0               PSI  6706.48  6809.30  6706.48  -68.41  -1.01%  11:35:59
1    PSEi Composite  6984.63  7109.75  6984.63   +0.38  +0.01%  02:58:59
